In [1]:
import mxnet as mx
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import numpy as np

# mxnet

In [33]:
idx_path = os.path.expanduser('~/workspace/MobileFaceNet_TF_i/datasets/faces_webface_112x112/train.idx')
rec_path = os.path.expanduser('~/workspace/MobileFaceNet_TF_i/datasets/faces_webface_112x112/train.rec')
imgrec = mx.recordio.MXIndexedRecordIO(idx_path, rec_path, 'r')

s = imgrec.read_idx(0)
header,_ = mx.recordio.unpack(s)
# HEADER(flag=2, label=array([490624., 501196.], dtype=float32), id=0, id2=0)
imgidx = list(range(1, int(header.label[0])))

# 图片数量
pic_number = int(header.label[0]) - 1
# 类别数量
class_number = int(header.label[1]) - int(header.label[0])

pics = []

for i, index in enumerate(imgidx):
    img_info = imgrec.read_idx(index)
    header, img_raw = mx.recordio.unpack(img_info)
    label = int(header.label)
    
    pics.append([label, img_raw])

print(len(pics)) # 490623

# tfrecord

In [4]:
def parse_function(example_proto):
    features = {'image_raw': tf.FixedLenFeature([], tf.string),
                'label': tf.FixedLenFeature([], tf.int64)}
    features = tf.parse_single_example(example_proto, features)
    # You can do more image distortion here for training data
    img = tf.image.decode_image(features['image_raw'])
    img = tf.reshape(img, shape=(112, 112, 3))

    #img = tf.py_func(random_rotate_image, [img], tf.uint8)
    img = tf.cast(img, dtype=tf.float32)
    img = tf.subtract(img, 127.5)
    img = tf.multiply(img,  0.0078125)
    img = tf.image.random_flip_left_right(img)
    label = tf.cast(features['label'], tf.int64)
    return img, label

In [ ]:
tfrecord_path = 'C:/Users/mengf/workspace/MobileFaceNet_TF_i/datasets/faces_webface_112x112/tfrecords/tran.tfrecords'
# test tfrecord
config = tf.ConfigProto(allow_soft_placement=True)
sess = tf.Session(config=config)
# training datasets api config
tfrecords_f = os.path.join(tfrecord_path)
dataset = tf.data.TFRecordDataset(tfrecords_f)
dataset = dataset.map(parse_function)
dataset = dataset.shuffle(buffer_size=20000)
dataset = dataset.batch(32)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
# begin iteration
for i in range(1000):
    sess.run(iterator.initializer)
    while True:
        try:
            images, labels = sess.run(next_element)
            for i in range(len(img)):
                images[i,...] = cv2.cvtColor(images[i, ...], cv2.COLOR_RGB2BGR)
            cv2.imshow('test', img)
            cv2.waitKey(0)
        except tf.errors.OutOfRangeError:
            print("End of dataset")

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
def my_generator(tfrecord_path='C:/Users/mengf/workspace/MobileFaceNet_TF_i/datasets/faces_webface_112x112/tfrecords/tran.tfrecords',batch_size=32,out_num=10572):
    """自定义generator
    
    # Argument
        tfrecord_path:
        batch_size
        out_num: 类别数量，用于生成onehot
        
    # Return
    
    """
    def parse_function(example_proto):
        features = {'image_raw': tf.FixedLenFeature([], tf.string),
                    'label': tf.FixedLenFeature([], tf.int64)}
        features = tf.parse_single_example(example_proto, features)
        # You can do more image distortion here for training data
        img = tf.image.decode_jpeg(features['image_raw'])
        img = tf.reshape(img, shape=(112, 112, 3))

        #img = tf.py_func(random_rotate_image, [img], tf.uint8)
        img = tf.cast(img, dtype=tf.float32)
        img = tf.subtract(img, 127.5)
        img = tf.multiply(img,  0.0078125)
        img = tf.image.random_flip_left_right(img)
        label = tf.cast(features['label'], tf.int64)
        label = tf.one_hot(label,out_num)
        return img, label
    
    config = tf.ConfigProto(allow_soft_placement=True)
    sess = tf.Session(config=config)
    # training datasets api config
    tfrecords_f = os.path.join(tfrecord_path)
    dataset = tf.data.TFRecordDataset(tfrecords_f)
    dataset = dataset.map(parse_function)
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_initializable_iterator()
    next_element = iterator.get_next()
    # begin iteration
    while(True):
        sess.run(iterator.initializer)
        while True:
            try:
                images, labels = sess.run(next_element)
                for i in range(len(images)):
                    images[i,...] = cv2.cvtColor(images[i, ...], cv2.COLOR_RGB2BGR)
                yield images,labels
            except tf.errors.OutOfRangeError:
#                 print("End of dataset")
                pass

def my_generator_wrapper():
    for image,label in my_generator():
        yield ([image,label],label)